## Work in Progress:

1.) Transfer model to full tensorflow implementation. Find a way to get maxout layer to work, also having trouble with latest tensorflow 1.4.0 release and using gpu

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: Michael Di Amore
"""

#Had to use python 2 for TF 1.4.0 was having problems with my virutal env
from __future__ import print_function
from __future__ import division

%matplotlib inline
import tensorflow as tf
import Query as query
import quandl 
import numpy as np
# np.random.seed(12345) # Set seed

import pandas as pd
from imblearn.combine import SMOTEENN 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pdb
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,recall_score,precision_score
import warnings
import matplotlib.pyplot as plt

import tensorflow as tf



## Data 

In [3]:
####
#  Load Data. If you don't want to / can't run the query.
# Basically this is all data from 2007-01-01 to 2017-10-06
load_data = True
if load_data == True:
    print ('Loading Data...')
    df_2001_2006 = pd.read_csv('Gdelt_events_20000101_20061231.csv')
    df_2001_2006 = df_2001_2006.set_index('sqldate',drop=True).sort_index()
    df_2007_2013 = pd.read_csv('Gdelt_events_20070101_20131231.csv')
    df_2007_2013 = df_2007_2013.set_index('sqldate',drop=True).sort_index()
    df_2014_01 = pd.read_csv('Gdelt_events_20140101_20140531.csv')
    df_2014_01  = df_2014_01 .set_index('sqldate',drop=True).sort_index()
    df_2014_02 = pd.read_csv('Gdelt_events_20140601_20141231.csv')
    df_2014_02  = df_2014_02 .set_index('sqldate',drop=True).sort_index()
    
    df_2015 = pd.read_csv('Gdelt_events_20150101_20151231.csv')
    df_2015 = df_2015.set_index('sqldate',drop=True).sort_index()
    df_2016_2017 = pd.read_csv('Gdelt_events_20160101_20171006.csv')
    df_2016_2017 = df_2016_2017.set_index('sqldate',drop=True).sort_index()
    gdelt_df = pd.concat([df_2001_2006,df_2007_2013,df_2014_01,df_2014_02,df_2015,df_2016_2017])
    
    del df_2001_2006,df_2007_2013,df_2014_01,df_2014_02,df_2015,df_2016_2017
    
####
# Set Params
lookback_window = int(np.round(252/2))
number_stdev = 2
daily_security_vol_target = .15/np.sqrt(252) #.15 is the 10 year SPY volalility found here
#http://performance.morningstar.com/funds/etf/ratings-risk.action?t=SPY&region=usa&culture=en-US
####

proj_id = 'capstone-v0'
start_date = '2000-01-01'
end_date = '2017-10-06'
ticker = '^GSPC'
my_query = query.query_tool(proj_id,start_date,end_date,ticker)
sql_query = """
            SELECT Actor1Name, GoldsteinScale,NumMentions,sourceurl,
            sqldate, avgtone, numarticles, numsources,  
            FROM [gdelt-bq:full.events] 
            WHERE sqldate > 20010101 and sqldate <= 20061231  and 
            Actor1Code like '%BUS%'and
            Actor1Geo_CountryCode like "%US%"
            """
if load_data == False:
    print ('Querying Gdelt...')
    my_query.query_gdelt(sql_query)
    my_query.gdelt_df = my_query.gdelt_df.set_index('sqldate',drop=True).sort_index()
    df = my_query.gdelt_df.copy(True)

#Creating Labels. i.e. if change in spx_return is x standard deviations
security_prices = my_query.query_yahoo()
security_return = np.log(security_prices['Adj Close']).diff() #log Return
security_vol = security_return.rolling(window=lookback_window).std().dropna()
security_mean = security_return.rolling(window=lookback_window).mean().dropna()
security_vol.name = 'Volatility'
security_return = security_return.loc[security_vol.index] #First entry is NAN because of return
# day_over_day_diff = np.abs(security_return.diff())#can subtract because of log returns
# event_idx = [(np.abs(security_mean)+(security_vol * 3.5)) < np.abs(security_return)]
event_idx = [((security_mean + security_vol * number_stdev) < security_return) | ((security_mean - security_vol *number_stdev) > security_return) ]
event_idx = np.array(event_idx).astype(int).flatten()
event_idx = pd.Series(event_idx,index=security_return.index)
print ('Data Loaded')

Loading Data...
Data Loaded


In [4]:
api_key =  'reqa36mksyfgx9BR6r88'
wti_co = my_query.query_quandl("FRED/DCOILWTICO",api_key)
unemploy = my_query.query_quandl("FRED/UNEMPLOY",api_key)
m1v = my_query.query_quandl("FRED/M1V",api_key)
m2v = my_query.query_quandl("FRED/M2V",api_key)
stressindex = my_query.query_quandl("FRED/STLFSI", api_key)
dff = my_query.query_quandl("FRED/DFF",api_key)
my_query.set_ticker('^VIX')
vix = my_query.query_yahoo()
vix = vix['Adj Close']
vix.name = 'VIX'
print ('Quandl Loaded')

#Use GLD as proxy for gold. Better to use the futures but don't have good continuous contracts
# my_query.set_ticker('GLD')
# gld = my_query.query_yahoo()
# gld = gld['Adj Close']

#Use Continuous Future of Front month gold contract as gold
# gold = my_query.query_quandl("SCF/CME_GC1_OR",api_key)
# gold = gold['Settle']
# #Use Continuous Future of front month  crude as oil
# oil = my_query.query_quandl("SCF/ICE_B1_OB",api_key)
# oil = oil['Settle']

Quandl Loaded


In [5]:
quandl_others = pd.concat([wti_co,unemploy,m1v,m2v,stressindex,vix,dff],axis=1)
quandl_others.columns = ['wit_co','unemploy','m1v','m2v','slsi','vix','dff']
quandl_others.ffill(inplace=True)
quandl_others.fillna(0,inplace=True)

In [6]:
def normalize_ts(series,lookback=lookback_window):
    rolling_mean = series.rolling(window=lookback).mean()
    rolling_std = series.rolling(window=lookback).std() + .10**3
    normalized = (series-rolling_mean)/rolling_std
    normalized = normalized.dropna()
    return(normalized)

In [8]:
#Collapse numerical data into x,y pairs by taking means
collapsed = gdelt_df.groupby(by=gdelt_df.index).mean()

#Shift data so as only to use yesterday's news for tomorrow's prediction
#i.e. we shift forward, using yesterday data as today
collapsed_shifted = collapsed.shift(1)
quandl_shifted = quandl_others.shift(1)

#Scale the data in such a way that we aren't looking forward into the feature
print ('Scaling using custom scaler')
collapsed_shifted = collapsed_shifted.apply(normalize_ts)
quandl_shifted = quandl_shifted.apply(normalize_ts)

collapsed_shifted.index = pd.to_datetime(collapsed_shifted.index,format='%Y%m%d')
collapsed_shifted = collapsed_shifted.loc[security_vol.index].dropna()
collapsed_shifted = pd.concat([collapsed_shifted,quandl_shifted],axis=1)
event_idx = event_idx.dropna()
collapsed_shifted = collapsed_shifted.loc[event_idx.index]

Scaling using custom scaler


In [9]:
print ('Creating train test split...')
X_train,X_test,Y_train,Y_test = train_test_split(collapsed_shifted,event_idx,stratify=None,test_size=.20,shuffle=False)
X_train,X_val,Y_train,Y_val = train_test_split(X_train,Y_train,test_size=.20,stratify=None,shuffle=False)

#Creating Copies of Data Frames these will be useful later for debugging
X_train_df = X_train.copy(True)
Y_train_df = Y_train.copy(True)
Y_val_df = Y_val.copy(True)
X_train  = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

Creating train test split...


In [10]:

pca = PCA(whiten=True)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
X_val = pca.transform(X_val)

print ('Length of X_train before smote')
print (len(X_train))
print ('Number of Positives before smote')
print (Y_train.sum())


print ('Performing Oversampling/Undersampling...')
s = SMOTEENN()
X_train,Y_train= s.fit_sample(X_train,Y_train)

print ('Length of X_train after smote')
print (len(X_train))

print ('Number of Positive samples after smote')
print (Y_train.sum())

# print ('Proportion after smote {}'.format(sum(Y_train)/len(Y_train))

X_train = X_train.reshape(-1,X_train.shape[1],1)
X_test = X_test.reshape(-1,X_test.shape[1],1)
X_val = X_val.reshape(-1,X_val.shape[1],1)
print ('Done!')

Length of X_train before smote
2780
Number of Positives before smote
166
Performing Oversampling/Undersampling...
Length of X_train after smote
4536
Number of Positive samples after smote
2574
Done!


## Tensorflow Version of Model

In [11]:
import tensorflow.contrib.keras as k
import tensorflow.contrib.slim as slim

In [12]:
D = X_train.shape[1]
Y_val = Y_val
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)
Y_train = Y_train.astype(np.int32)
Y_val = Y_val.astype(np.int32)
features = {'x':X_train}
drop_rate = .55

In [18]:
#Had to use a separate function couldn't get TF GPU to work with TF 1.4.0
#Taken from 

def max_out(inputs, num_units, axis=None):
    shape = inputs.get_shape().as_list()
    if shape[0] is None:
        shape[0] = -1
    if axis is None:  # Assume that channel is the last dimension
        axis = -1
    num_channels = shape[axis]
    if num_channels % num_units:
        raise ValueError('number of features({}) is not '
                         'a multiple of num_units({})'.format(num_channels, num_units))
    shape[axis] = num_units
    shape += [num_channels // num_units]
    outputs = tf.reduce_max(tf.reshape(inputs, shape), -1, keep_dims=False)
    return outputs

In [30]:
def tf_model(features,labels,mode):
    with tf.device('/gpu:0'):
        if tf.estimator.ModeKeys.TRAIN == 'train':
            is_training=True
        else:
            is_training = False
        layer1 =  tf.layers.conv1d(inputs=features['x'],name='x',filters=512,kernel_size=5,padding='same',
                                                       activation=tf.nn.relu,kernel_initializer=k.initializers.he_normal())
        layer1 =  tf.contrib.layers.batch_norm(layer1, is_training=is_training, decay=0.999)
        layer1 = tf.layers.dropout(inputs=layer1,training=is_training,rate=drop_rate)


        layer2 =  tf.layers.conv1d(layer1,filters=256,kernel_size=5,padding='same',
                                   activation=tf.nn.relu,kernel_initializer=k.initializers.he_normal())
        layer2 =  tf.contrib.layers.batch_norm(layer2, is_training=is_training, decay=0.999)
        layer2 = tf.layers.dropout(inputs=layer2,training=is_training,rate=drop_rate)


        layer3 =  tf.layers.conv1d(layer2,filters=128,kernel_size=5,padding='same',activation=tf.nn.relu,kernel_initializer=k.initializers.he_normal())
        layer3 =  tf.contrib.layers.batch_norm(layer3, is_training=is_training, decay=0.999)
        layer3 = tf.layers.dropout(inputs=layer1,training=is_training,rate=drop_rate)


        layer4 =  tf.layers.conv1d(layer3,filters=128,kernel_size=3,padding='same',activation=tf.nn.relu,kernel_initializer=k.initializers.he_normal())
        layer4 =  tf.contrib.layers.batch_norm(layer4, is_training=is_training, decay=0.999)
        layer4 = tf.layers.dropout(inputs=layer4,training=is_training,rate=drop_rate)


        layer5 =  tf.layers.conv1d(layer4,filters=128,kernel_size=3,padding='same',activation=tf.nn.relu,kernel_initializer=k.initializers.he_normal())
        layer5 =  tf.contrib.layers.batch_norm(layer5, is_training=is_training, decay=0.999)
        layer5 = tf.layers.dropout(inputs=layer5,training=is_training,rate=drop_rate)


        layer6 =  tf.layers.conv1d(layer5,filters=128,kernel_size=3,padding='same',activation=tf.nn.relu,kernel_initializer=k.initializers.he_normal())
        layer6 =  tf.contrib.layers.batch_norm(layer6, is_training=is_training, decay=0.999)
        layer6 = tf.layers.dropout(inputs=layer6,training=is_training,rate=drop_rate)

        flatten = tf.contrib.layers.flatten(inputs=layer6)
        flat_shape = flatten.get_shape().as_list()[1]
        
        layer7 =  max_out(flatten,flat_shape-1024)
        layer7 =  tf.contrib.layers.batch_norm(layer7, is_training=is_training, decay=0.999)
    #     layer7 = tf.layers.dropout(inputs=layer7,training=is_training,rate=drop_rate)

        layer8 =  max_out(layer7,512)
        layer8 =  tf.contrib.layers.batch_norm(layer8, is_training=is_training, decay=0.999)
    #     layer8 = tf.layers.dropout(inputs=layer8,training=is_training,rate=drop_rate)

        layer9 =  max_out(layer8,256)
        layer9 =  tf.contrib.layers.batch_norm(layer9, is_training=is_training, decay=0.999)
    #     layer9 = tf.layers.dropout(inputs=layer9,training=is_training,rate=drop_rate)

        logits = tf.layers.dense(layer9,units=2)
        

        predictions = {
          "classes": tf.argmax(input=logits, axis=1),
          "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
            }

        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

          # Calculate Loss (for both TRAIN and EVAL modes)
        ohc =  tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=ohc, logits=logits)

        # Configure the Training Op (for TRAIN mode)
        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
            train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

        # Add evaluation metrics (for EVAL mode)
        eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

sess = tf.InteractiveSession()

my_classifier = tf.estimator.Estimator(
    model_fn=tf_model)
    
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(X_train)},
      y=np.array(Y_train),
      num_epochs=None,
      shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_val},
    y=Y_val,
    num_epochs=1,
    shuffle=False)


my_classifier.train(input_fn=train_input_fn, steps=2000)
eval_results = my_classifier.evaluate(input_fn=eval_input_fn)

predictions = list(my_classifier.predict(input_fn=eval_input_fn))
predicted_classes = [p["classes"] for p in predictions]
print(predicted_classes)

    


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_model_dir': '/tmp/tmpcm98o7mo', '_save_summary_steps': 100, '_session_config': None, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000}


ValueError: number of features(128) is not a multiple of num_units(1024)

In [28]:
acc = accuracy_score(Y_val,predicted_classes)
f1 = f1_score(Y_val,predicted_classes)
roc = roc_auc_score(Y_val,predicted_classes)
prec = precision_score(Y_val,predicted_classes)
recall= recall_score(Y_val,predicted_classes)


print (' Standard Deviations: {}'.format(number_stdev))
print (' Accuracy: {}'.format(acc))
print (' F1: {}'.format(f1))
print (' RoC: {}'.format(roc))
print (' Precision: {}'.format(prec))
print (' Recall: {}'.format(recall))

 Standard Deviations: 2
 Accuracy: 0.42589928057553955
 F1: 0.111358574610245
 RoC: 0.5437648895095704
 Precision: 0.06067961165048544
 Recall: 0.6756756756756757


[0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
